# பெயரிடப்பட்ட பொருள் அடையாளம் (NER)

இந்த நோட்புக் [AI for Beginners Curriculum](http://aka.ms/ai-beginners) இலிருந்து பெறப்பட்டது.

இந்த உதாரணத்தில், [Annotated Corpus for Named Entity Recognition](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus) என்ற Kaggle தரவுத்தொகுப்பைப் பயன்படுத்தி NER மாடலை எப்படி பயிற்சி செய்யலாம் என்பதை நாம் கற்றுக்கொள்வோம். தொடருவதற்கு முன், [ner_dataset.csv](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus?resource=download&select=ner_dataset.csv) கோப்பை தற்போதைய அடைவில் பதிவிறக்கவும்.


In [62]:
import pandas as pd
from tensorflow import keras
import numpy as np

## தரவுத்தொகுப்பை தயாரித்தல்

முதலில், தரவுத்தொகுப்பை ஒரு dataframe-இல் படிக்க ஆரம்பிப்போம். Pandas-ஐப் பயன்படுத்துவது குறித்து மேலும் அறிய விரும்பினால், எங்கள் [தரவியல் அறிவியல் தொடக்கத்திற்கான](http://aka.ms/datascience-beginners) [தரவு செயலாக்க பாடத்தினை](https://github.com/microsoft/Data-Science-For-Beginners/tree/main/2-Working-With-Data/07-python) பார்வையிடுங்கள்.


In [3]:
df = pd.read_csv('ner_dataset.csv',encoding='unicode-escape')
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


நாம் தனித்துவமான குறிச்சொற்களை பெறுவோம் மற்றும் குறிச்சொற்களை வகுப்பு எண்களாக மாற்ற பயன்படுத்தக்கூடிய தேடல் அகராதிகளை உருவாக்குவோம்:


In [4]:
tags = df.Tag.unique()
tags

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [8]:
id2tag = dict(enumerate(tags))
tag2id = { v : k for k,v in id2tag.items() }

id2tag[0]

'O'

இப்போது நாங்கள் சொற்கள்தொகுப்புடன் அதே செயல்பாட்டை செய்ய வேண்டும். எளிமைக்காக, நாங்கள் சொற்களின் அடிக்கடி பயன்பாட்டை கணக்கில் எடுத்துக்கொள்ளாமல் சொற்கள்தொகுப்பை உருவாக்குவோம்; உண்மையான வாழ்க்கையில், நீங்கள் Keras வெக்டரைசரை பயன்படுத்தி, சொற்களின் எண்ணிக்கையை வரையறுக்க விரும்பலாம்.


In [14]:
vocab = set(df['Word'].apply(lambda x: x.lower()))
id2word = { i+1 : v for i,v in enumerate(vocab) }
id2word[0] = '<UNK>'
vocab.add('<UNK>')
word2id = { v : k for k,v in id2word.items() }

பயிற்சிக்காக வாக்கியங்களின் தரவுத்தொகுப்பை உருவாக்க வேண்டும். முதன்மை தரவுத்தொகுப்பில் உள்ள அனைத்து தனிப்பட்ட வாக்கியங்களையும் `X` (சொற்களின் பட்டியல்) மற்றும் `Y` (டோக்கன்களின் பட்டியல்) என பிரிக்கலாம்:


In [41]:
X,Y = [],[]
s,t = [],[]
for i,row in df[['Sentence #','Word','Tag']].iterrows():
    if pd.isna(row['Sentence #']):
        s.append(row['Word'])
        t.append(row['Tag'])
    else:
        if len(s)>0:
            X.append(s)
            Y.append(t)
        s,t = [row['Word']],[row['Tag']]
X.append(s)
Y.append(t)


நாம் இப்போது அனைத்து வார்த்தைகள் மற்றும் டோக்கன்களை வெக்டர்மயமாக்குவோம்:


In [93]:
def vectorize(seq):
    return [word2id[x.lower()] for x in seq]

def tagify(seq):
    return [tag2id[x] for x in seq]

Xv = list(map(vectorize,X))
Yv = list(map(tagify,Y))

Xv[0], Yv[0]

([10386,
  23515,
  4134,
  29620,
  7954,
  13583,
  21193,
  12222,
  27322,
  18258,
  5815,
  15880,
  5355,
  25242,
  31327,
  18258,
  27067,
  23515,
  26444,
  14412,
  358,
  26551,
  5011,
  30558],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0])

எளிமைக்காக, நாம் அனைத்து வாக்கியங்களையும் அதிகபட்ச நீளத்திற்கு 0 டோக்கன்களுடன் நிரப்புவோம். நிஜ வாழ்க்கையில், நாம் மேலும் புத்திசாலித்தனமான உத்தியைப் பயன்படுத்த விரும்பலாம், மற்றும் ஒவ்வொரு மினிபேட்சிற்குள் மட்டுமே வரிசைகளை நிரப்பலாம்.


In [51]:
X_data = keras.preprocessing.sequence.pad_sequences(Xv,padding='post')
Y_data = keras.preprocessing.sequence.pad_sequences(Yv,padding='post')

## டோக்கன் வகைப்பாட்டுக்கான நெட்வொர்க் வரையறுத்தல்

டோக்கன் வகைப்பாட்டுக்காக இரண்டு அடுக்கு இருதிசை LSTM நெட்வொர்க் பயன்படுத்தப் போகிறோம். கடைசி LSTM அடுக்கின் ஒவ்வொரு வெளியீட்டிலும் அடர்த்தியான வகைப்பாட்டாளரை பயன்படுத்த, `TimeDistributed` கட்டமைப்பை பயன்படுத்துவோம், இது LSTM இன் ஒவ்வொரு படியிலும் அதே அடர்த்தியான அடுக்கை நகலெடுக்கிறது:


In [94]:
maxlen = X_data.shape[1]
vocab_size = len(vocab)
num_tags = len(tags)
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, 300, input_length=maxlen),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.TimeDistributed(keras.layers.Dense(num_tags, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 104, 300)          9545400   
                                                                 
 bidirectional_6 (Bidirectio  (None, 104, 200)         320800    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 104, 200)         240800    
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, 104, 17)          3417      
 tributed)                                                       
                                                                 
Total params: 10,110,417
Trainable params: 10,110,417
Non-trainable params: 0
__________________________________________

இங்கே, நாங்கள் எங்கள் தரவுத்தொகுப்புக்கான `maxlen` ஐ தெளிவாக குறிப்பிடுகிறோம் - நெட்வொர்க் மாறுபட்ட நீள வரிசைகளை கையாள விரும்பினால், நெட்வொர்க் வரையறை செய்யும்போது நாங்கள் மேலும் சிக்கலாக இருக்க வேண்டும்.

இப்போது மாடலை பயிற்றுவிக்கலாம். வேகத்திற்காக, நாங்கள் ஒரு எபோக்கிற்காக மட்டுமே பயிற்சி அளிக்கிறோம், ஆனால் நீண்ட நேரத்திற்கு பயிற்சி அளிக்க முயற்சிக்கலாம். மேலும், நீங்கள் பயிற்சி தரவுத்தொகுப்பாக சில பகுதியை பிரிக்க விரும்பலாம், சரிபார்ப்பு துல்லியத்தை கவனிக்க.


In [57]:
model.fit(X_data,Y_data)

1499/1499 [==============================] - 740s 488ms/step - loss: 0.0667 - acc: 0.9841


## முடிவை சோதனை செய்வது

இப்போது, எங்கள் entity recognition மாடல் ஒரு மாதிரி வாக்கியத்தில் எப்படி செயல்படுகிறது என்பதை பார்ப்போம்:


In [91]:
sent = 'John Smith went to Paris to attend a conference in cancer development institute'
words = sent.lower().split()
v = keras.preprocessing.sequence.pad_sequences([[word2id[x] for x in words]],padding='post',maxlen=maxlen)
res = model(v)[0]

In [92]:
r = np.argmax(res.numpy(),axis=1)
for i,w in zip(r,words):
    print(f"{w} -> {id2tag[i]}")

john -> B-per
smith -> I-per
went -> O
to -> O
paris -> B-geo
to -> O
attend -> O
a -> O
conference -> O
in -> O
cancer -> B-org
development -> I-org
institute -> I-org


## முக்கிய குறிப்புகள்

எளிய LSTM மாடல் கூட NER-இல் நியாயமான முடிவுகளை வழங்குகிறது. ஆனால், மேலும் சிறந்த முடிவுகளை பெற, BERT போன்ற பெரிய முன்பதிவுசெய்யப்பட்ட மொழி மாடல்களை பயன்படுத்த விரும்பலாம். Huggingface Transformers நூலகத்தைப் பயன்படுத்தி NER-க்கு BERT-ஐ பயிற்சி செய்வது [இங்கே](https://huggingface.co/course/chapter7/2?fw=pt) விளக்கப்பட்டுள்ளது.



---

**குறிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையை பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. எங்கள் நோக்கம் துல்லியமாக இருக்க வேண்டும் என்பதுதான், ஆனால் தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது துல்லியமின்மைகள் இருக்கக்கூடும் என்பதை தயவுசெய்து கவனத்தில் கொள்ளவும். அதன் தாய்மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பல்ல.
